In [24]:
import numpy as np
import os

from utils import set_params
from utils import load_pickle, extract_used_data
from utils import mergeAB, align_track, z_score_on
from utils import pca_fit
from utils import JPCA
from utils import plot_neuron_id

from utils.config import Params
import math
from typing import List, Dict, Any, Iterable, Optional
import matplotlib.pyplot as plt

In [25]:
def get_data(file_path: str, params: Params) -> Dict[str, Any]:
    data = load_pickle(file_path)
    data = extract_used_data(data)
    mergeAB(data)
    align_track(data, params)
    for idx in params.ana_index_grid(params.tt, params.bt):
        z_score_on(data, params, [params.tt[idx[0]]], [params.bt[idx[1]]])
    return data

In [26]:
# Params set

params = set_params(tt_preset='merge',
                         bt_preset='basic',
                         len_pos_average=10,
                         gaussian_range=10,
                         gaussian_sigma=10)


In [27]:

data = get_data('../../data/first_to_pat/RB04-PFCsep.pkl', params)


In [28]:
import ipywidgets as widgets
from IPython.display import display, clear_output

def browse_neurons(data, params, ana_tt=['*'], ana_bt=['correct'], id_min=0, id_max=None):
    # 估一个最大 id（按你数据结构自己改一下）
    # 这里假设 aligned_firing[index] 是 (num_neurons, len_track)
    # ana_tt/ana_bt 选一个 index 来拿 num_neurons
    for index in params.ana_index_grid(ana_tt, ana_bt):
        fr = data["aligned_firing"][index]
        if fr is not None:
            n_neurons = fr.shape[0]
            break
    else:
        raise ValueError("No valid fr found for given ana_tt/ana_bt")

    if id_max is None:
        id_max = n_neurons - 1

    slider = widgets.IntSlider(value=id_min, min=id_min, max=id_max, step=1,
                               description='Neuron id', continuous_update=False)
    btn_prev = widgets.Button(description='◀ Prev', layout=widgets.Layout(width='80px'))
    btn_next = widgets.Button(description='Next ▶', layout=widgets.Layout(width='80px'))
    out = widgets.Output()

    def render(_=None):
        with out:
            clear_output(wait=True)
            plot_neuron_id(data, params, ana_tt=ana_tt, ana_bt=ana_bt, id=slider.value)

    def on_prev(_):
        slider.value = max(slider.min, slider.value - 1)

    def on_next(_):
        slider.value = min(slider.max, slider.value + 1)

    btn_prev.on_click(on_prev)
    btn_next.on_click(on_next)
    slider.observe(render, names='value')

    display(widgets.HBox([btn_prev, slider, btn_next]), out)
    render()

# 使用
browse_neurons(data, params, ana_tt=['pattern*'], ana_bt=['correct'])


Output()